# Développement d’un module de classification d’images IHC par IA

### By Maëlle Broustal, Rafik Mankour & Ian Pellet

## Import

In [1]:
import numpy as np
import pandas as pd
import skimage
import skimage.draw
import skimage.measure
import matplotlib.pyplot as plt
from xml.dom import minidom as minidom
import PIL 

In [3]:
#from tensorflow import keras as k

In [4]:
k.__version__

NameError: name 'k' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### Openslide

In [ ]:
!apt update && apt install -y openslide-tools
!pip install openslide-python

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://ppa

In [5]:
import openslide

## Annotation

L'annotion est faite sous ce format : (les ... représentent un skip)
```
<Annotations><Annotation LineColor="65535" Name="Layer 1" Visible="True"><Regions><Region Type="Polygon" HasEndcaps="0" 
NegativeROA="0"><Vertices><V X="2667" Y="16944" /><V X="2667" Y="16967" />
<V X="2667" Y="17058" /><V X="2576" Y="17422" /><V X="2440" Y="18104" /><V X="2394" Y="18445" /><V X="2394" Y="18627" /><V X="2394" Y="18900" />
<V X="2394" Y="19286" /><V X="2394" Y="19696" /><V X="2394" Y="20082" /><V X="2303" Y="20423" /><V X="2281" Y="20651" /><V X="2190" Y="20946" />
<V X="2099" Y="21242" /><V X="2112" Y="21410" /><V X="2112" Y="21545" /><V X="2090" Y="21613" /><V X="2090" Y="21725" /><V X="2090" Y="21793" />
<V X="2090" Y="21838" /> ... <V X="2090" Y="21928" /></Vertices></Region> </Regions></Annotation></Annotations>

```

### etree.ElementTree

#### Extraction de vertex

Méthode 1:
trouver la fonction qui le fait déjà à notre place

Méthode 2:
* pour récupérer les points:
  * récupérer l'annotation comme un string
  * extraire name, NegativeROA
  * extraire chaque coordonnée: liste de vertices? X et Y
* pour utiliser la fonction polygon:
  * créer les vecteurs d'un point au suivant (nécessité de notion d'ordre -> array & df ok)
  * c'est tout

In [ ]:
import xml.etree.ElementTree as ET

tree = ET.parse('annot.annotations')
root = tree.getroot()

# one specific item attribute
print('Item #2 attribute:')
print(root[0][0][0][0][0].attrib)
#root[annotations][annotation][régions][région][sommet]

# all item attributes
print('\nAll attributes:')
for elem in root:
    for subelem in elem:
        print(subelem.attrib)

# one specific item's data
print('\nItem #2 data:')
print(root[0].text)

# all items data
print('\nAll item data:')
for elem in root:
    for subelem in elem:
        print(subelem.text)

FileNotFoundError: ignored

In [ ]:
len(root), len(root[0]), len(root[0][0]), len(root[0][0][0]), len(root[0][0][0][0]), len(root[0][0][0][0][0])

NameError: ignored

In [ ]:
# liste of vertices:

#liste de polygones avec tous leurs vertices
#liste de listes (les polygones) d'objets (les vertices)
# [ [régionA-vertice1, régionA-vertice2, régionA-verticeplein], [régionB-vertice 1, régionB-vertice2, régionB-verticebcp...]]

print('\nAll vertices:')

vertices_list = [[]for i in range(len(root[0][0]))]
for annotation in root: #normalement y a qu'une
  for regions in annotation: #normalement y en a qu'une
    for n, region in enumerate(regions):
      print("new region")
      print(n)
      for vertices in region:
        for vertice in vertices:
          vertices_list[n].append(vertice.attrib)

vertices_list

### dom.minidom

#### Extraction vertex d'une annotation

In [ ]:
#pour Rafik
annot = minidom.parse("/content/drive/MyDrive/data/annot.annotations") # open XML file

In [ ]:
#pour Mallou
annot = minidom.parse("/content/drive/MyDrive/Projet 4BIM/data/annot.annotations") # open XML file

In [ ]:
regions = annot.getElementsByTagName("Region") # list of regions in the annotation file
print(len(regions))

26


In [ ]:
V = [] # list of vertices in each region
V_coord = [] # list of coord of each vertex in each region
for r in regions :
  if r.getAttribute("NegativeROA")=='0': 
    vertices = r.getElementsByTagName("V")
    V.append(vertices)
    
    v_coord = []
    for v in vertices :
        coord = {"X": v.getAttribute('X'), "Y": v.getAttribute('Y')}
        v_coord.append(coord)
    V_coord.append(v_coord)

#### 0/1 array representation of regions of interest

In [ ]:
img = np.ones((8064, 6720)) # creates an array of ones of the same shape of our image
ds_rate = 8 # downsample rate

polygons = [] # list of polygons, each object is an array containing the coord of the px in a region of V_coord

for region in V_coord:
  x = [] # list of x coord in the region
  y = [] # list of y coord in the region
  
  for v in region:
    # as we dont take the full resolution slide of the image, we need to divide the coord of each vertice by the downsampling rate
    temp_x = int(v['X'])/ds_rate 
    temp_y = int(v['Y'])/ds_rate
    
    x.append(int(temp_x))
    y.append(int(temp_y))

  # sikimage computes the coord of each px in the region from the vertices defining the perimeter of the region
  polygons.append(skimage.draw.polygon(x,y)) 

In [ ]:
# modify img so that each px in a region of V_coord is set to 0
for p in range(len(polygons)):
  poly = np.transpose(polygons[p])

  for i in range(len(poly)):
    img[(poly[i][0], poly[i][1])] = 0

#### Passage de la matrice au vertex_list

In [ ]:
def neighbors(point):
  x = point.get("X")
  y = point.get("Y")
  #n1={"X":x-1, "Y":y-1}
  #neighbors_list=[n1]
  neighbors_list = [{"X": x-1, "Y": y-1},{"X":x-1, "Y":y},{"X":x-1, "Y":y+1}, {"X":x, "Y":y-1}, {"X":x, "Y":y+1}, {"X":x+1, "Y":y-1}, {"X":x+1, "Y":y}, {"X":x+1, "Y":y+1}]
  return neighbors_list

In [ ]:
def isolation(matrix):

  regions = [] #liste des régions
  nb_r = 0 #nombre de régions

  for x, line in enumerate(matrix):
    for y, value in enumerate(line):

      if value==1: 
        print 
        new_point = {"X":x, "Y":y}
        #une fois qu'on a trouvé un point d'intérêt, on vérifie qu'il n'est pas déjà dans une région
        already = False 
        for region in regions:
          if new_point in region:
            already = True

        if already == False: 
          #s'il n'est nulle part, on crée une nouvelle région          
          regions.append([])
          actual_r = nb_r    # je sais pas comment conserver le numéro de la région où on est, alors le voilà
          nb_r += 1          # ça c'est pour plus tard

          #puis on regarde ses voisins: pour ça, on l'ajoute à la liste-d'attente-des-points
          waiting = []
          waiting.append(new_point)

          #et on travaille ensuite à partir de la liste d'attente
          for point in waiting:
            if point.get("X") >= 0 and point.get("Y") >= 0 and point.get("X")<=len(matrix)-1 and point.get("Y")<=len(line)-1 and matrix[point.get("X")][point.get("Y")] == 1:
              if point not in regions[actual_r]:
                regions[actual_r].append(point)
                for neighbor in neighbors(point):
                  #is this point already in regions[actual_r]?
                  if neighbor not in regions[actual_r]:
                    waiting.append(neighbor)

  return regions

In [ ]:
def matrix_to_vertices(matrix):
 
  regions = isolation(matrix)
  vertices=[]

  for r, region in enumerate(regions):
    vertices.append([])

    for point_of_interest in region:
      x = point_of_interest.get("X")
      y = point_of_interest.get("Y")

      if x==0 or y==0 or x==len(matrix)-1 or y==len(line)-1:
        contour = True #le point appartient à un "bord" de la matrice

      elif sum([matrix[x-1][y-1], matrix[x-1][y], matrix[x-1][y-1], matrix[x][y-1], matrix[x][y+1], matrix[x+1][y+1], matrix[x+1][y-1], matrix[x+1][y]])<=5:
        contour = True #le point est un contour hors bordure

      if contour == True:
        vertices[r].append(point_of_interest)

  return vertices 

In [ ]:
matrix = [[1,1,1,0,0],[1,1,0,0,0],[0,0,0,1,1],[0,0,1,1,1],[0,1,1,1,0]]

In [ ]:
matrix_to_vertices(matrix)

[[{'X': 0, 'Y': 0},
  {'X': 0, 'Y': 1},
  {'X': 1, 'Y': 0},
  {'X': 1, 'Y': 1},
  {'X': 0, 'Y': 2}],
 [{'X': 2, 'Y': 3},
  {'X': 2, 'Y': 4},
  {'X': 3, 'Y': 2},
  {'X': 3, 'Y': 3},
  {'X': 3, 'Y': 4},
  {'X': 4, 'Y': 1},
  {'X': 4, 'Y': 2},
  {'X': 4, 'Y': 3}]]

#### Création nouvelle annotation (vertex -> xml)

On veut d'abord convertir la matrice de coordonnées données par l'outil de classification en liste de vertices V_coord nécessaire à la composition du fichier xml.

In [ ]:
################ MARCHE PAS ENCORE

coord = np.array([[1, 1, 0, 1, 1], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0]])

V_coord = [] # Pour convertir une matrice en liste de coordonnées
for sub1, sub2 in coord: 
      
    # zip() used to form pairing 
    for ele in zip(sub1, sub2): 
        V_coord.append(ele) 

ValueError: ignored

In [ ]:
new_annot = minidom.Document() # create a new file

annotations = new_annot.createElement("Annotations") # create element Annotations

annotation = new_annot.createElement("Annotation") # create element Annotation
# set all the attributes 
annotation.setAttribute('LineColor',"65535") 
annotation.setAttribute('Name', "Layer 1")
annotation.setAttribute('Visible', "True")

annotations.appendChild(annotation) # add Annotation to Annotations

<DOM Element: Annotation at 0x7f1cea674eb0>

In [ ]:
for R in V_coord:
    new_region = new_annot.createElement("Region") # create a new region
    for V in R:
        new_vertice = new_annot.createElement("V") # create a new vertice
        # set vertice's coord
        new_vertice.setAttribute('X', V['X'])
        new_vertice.setAttribute('Y', V['Y'])
        
        new_region.appendChild(new_vertice) # add vertice to region
        
    annotation.appendChild(new_region) # add region to annotation

In [ ]:
new_annot.appendChild(annotations) # add Annotations to our document

<DOM Element: Annotations at 0x7f1cea674b90>

In [ ]:
myfile = open("annot_new.annotations", "w") # open a new file
myfile.write(new_annot.toprettyxml()) # write the content of our document in XML in the file
myfile.close()

## Chargement image

In [ ]:
img_test = openslide.OpenSlide('./data/CRC1.ndpi') # open whole-slide image

NameError: ignored

In [ ]:
#Mallou
img_test = openslide.OpenSlide('/content/drive/MyDrive/Projet 4BIM/data/CRC1.ndpi')

In [ ]:
img_test.level_dimensions # dimensions for each levels of the slide

((64512, 53760),
 (32256, 26880),
 (16128, 13440),
 (8064, 6720),
 (4032, 3360),
 (2016, 1680),
 (1008, 840),
 (504, 420),
 (252, 210),
 (126, 105))

In [ ]:
img_test.level_downsamples # downsample rates of each level

(1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0, 256.0, 512.0)

In [ ]:
img_test.get_thumbnail((1200, 1200)) # RGB image of the slide of size (1200,1200)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# for each level, print level and size and open a region of the slide
max_size = (1800, 1000)
for lvl in range(img_test.level_count):
    lvl_size = img_test.level_dimensions[lvl]
    print(lvl, lvl_size)
    disp_size = max_size if lvl_size > max_size else lvl_size

    img_test.read_region((0,0), lvl, disp_size).show()

0 (64512, 53760)
1 (32256, 26880)
2 (16128, 13440)
3 (8064, 6720)
4 (4032, 3360)
5 (2016, 1680)
6 (1008, 840)
7 (504, 420)
8 (252, 210)
9 (126, 105)


In [ ]:
# we can extract a region of a level of our slide 
PIL_img = img_test.read_region((8000,4000), 5, (1200, 1200))
type(PIL_img)

PIL.Image.Image

In [ ]:
np_img = np.array(PIL_img) # convert PIL Image to a Numpy array
np_img.shape # RGBA

(1200, 1200, 4)

In [ ]:
lvl = 5 # levels 0 to 2 are to big 
full_img = img_test.read_region((0,0), lvl, img_test.level_dimensions[lvl])
full_np = np.array(full_img)
full_np.shape

(1680, 2016, 4)

In [ ]:
img_test.close() # close image